# Distance vs Similarity Matrix Time

In [1]:
# Torch
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch import optim 


# Train + Data 
import sys 
sys.path.append('../Layers')
from Conv1d_NN import *
from Conv2d_NN import *

from Conv1d_NN_spatial import * 
from Conv2d_NN_spatial import * 

sys.path.append('../Data')
from CIFAR10 import * 


sys.path.append('../Train')
from train2d import * 


/Users/mingikang/miniforge3/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Distance & Similarity Models

In [2]:
# ConNN 2D All Sample Model with K = 9 (Distance Matrix)
Conv2dNN_9_all_distance = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
      magnitude_type="distance"
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
      magnitude_type="distance"

   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_all_distance, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 12, 256]               0
            Conv1d-2              [-1, 64, 256]           6,976
         Conv1d_NN-3              [-1, 64, 256]               0
         Conv2d_NN-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 64, 256]               0
            Conv1d-7             [-1, 128, 256]          73,856
         Conv1d_NN-8             [-1, 128, 256]               0
         Conv2d_NN-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

In [3]:
# ConNN 2D All Sample Model with K = 9 (Distance Matrix)
Conv2dNN_9_all_similarity = nn.Sequential(
   Conv2d_NN(
      in_channels=3,
      out_channels=16,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
      magnitude_type="similarity"
   ), 
   nn.ReLU(),

   Conv2d_NN(
      in_channels=16,
      out_channels=32,
      K=9,
      stride=9,
      padding=0,
      shuffle_pattern="BA", 
      shuffle_scale=2, 
      samples="all", 
      magnitude_type="similarity"

   ),
   nn.ReLU(),
   
   nn.Flatten(),
   nn.Linear(32768, 1024),
   nn.ReLU(),
   nn.Linear(1024, 10)
).to('cpu')

from torchsummary import summary
summary(Conv2dNN_9_all_similarity, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1              [-1, 12, 256]               0
            Conv1d-2              [-1, 64, 256]           6,976
         Conv1d_NN-3              [-1, 64, 256]               0
         Conv2d_NN-4           [-1, 16, 32, 32]               0
              ReLU-5           [-1, 16, 32, 32]               0
           Flatten-6              [-1, 64, 256]               0
            Conv1d-7             [-1, 128, 256]          73,856
         Conv1d_NN-8             [-1, 128, 256]               0
         Conv2d_NN-9           [-1, 32, 32, 32]               0
             ReLU-10           [-1, 32, 32, 32]               0
          Flatten-11                [-1, 32768]               0
           Linear-12                 [-1, 1024]      33,555,456
             ReLU-13                 [-1, 1024]               0
           Linear-14                   

# II. Distance & Similarity Time Evaluation

In [4]:
cifar10 = CIFAR10()

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Distance
Conv2dNN_9_all_distance.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_all_distance.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_all_distance, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_all_distance, cifar10.test_loader)

Epoch 1, Time: 54.59993100166321, Loss: 1.5074244826041219
Epoch 2, Time: 54.776877880096436, Loss: 1.168537397518792
Epoch 3, Time: 53.591310024261475, Loss: 0.9431182021832527
Epoch 4, Time: 53.18561577796936, Loss: 0.7015001652643199
Epoch 5, Time: 53.244892835617065, Loss: 0.4671285363852673
Epoch 6, Time: 53.23835206031799, Loss: 0.30799558736822186
Epoch 7, Time: 53.321255922317505, Loss: 0.22136077013276423
Epoch 8, Time: 53.240829944610596, Loss: 0.1840662612744114
Epoch 9, Time: 52.90734505653381, Loss: 0.15408209330566666
Epoch 10, Time: 53.52429223060608, Loss: 0.13070755098205622

 Average epoch time: 53.563070273399354
Accuracy on test set: 55.34%


55.34

In [6]:
# Similarity
Conv2dNN_9_all_similarity.to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Conv2dNN_9_all_similarity.parameters(), lr=0.001)
num_epochs = 10 
train_model(Conv2dNN_9_all_similarity, cifar10.train_loader, criterion, optimizer, num_epochs)
evaluate_accuracy(Conv2dNN_9_all_similarity, cifar10.test_loader)

Epoch 1, Time: 53.403226137161255, Loss: 1.5659344574374616
Epoch 2, Time: 52.248836278915405, Loss: 1.231438933812139
Epoch 3, Time: 52.17728900909424, Loss: 1.0457029331980459
Epoch 4, Time: 52.100014209747314, Loss: 0.8662748996101682
Epoch 5, Time: 52.000330209732056, Loss: 0.6686297812882591
Epoch 6, Time: 52.07300305366516, Loss: 0.4838443547106155
Epoch 7, Time: 52.02247500419617, Loss: 0.3422462342835753
Epoch 8, Time: 52.02267599105835, Loss: 0.2563483023639683
Epoch 9, Time: 52.20414686203003, Loss: 0.20328128369301177
Epoch 10, Time: 51.97289490699768, Loss: 0.17105288792144308

 Average epoch time: 52.22248916625976
Accuracy on test set: 54.51%


54.51

## III. Profiler for Distance vs. Similarity

In [7]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Conv2d
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

### Distance Profiler

In [8]:
input = torch.randn(32, 3, 32, 32)

Conv2dNN_9_all_distance.to('cpu')

with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True) as prof1:
    with record_function("model_inference"):
        Conv2dNN_9_all_distance(input)

print(prof1.key_averages().table(sort_by="cpu_time_total"))

-------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      model_inference         6.65%       5.411ms       100.00%      81.338ms      81.338ms      42.31 Mb     -83.81 Mb             1  
                         aten::conv1d         0.02%      20.000us        44.25%      35.989ms      17.994ms       6.00 Mb           0 b             2  
                    aten::convolution         0.11%      91.000us        44.22%      35.969ms      17.985ms       6.00 Mb           0 b             2  
                   aten::_convolution         1.42%       1.153ms        44.11%      35.

STAGE:2025-02-09 13:43:39 4485:152408 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-09 13:43:39 4485:152408 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-09 13:43:39 4485:152408 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


### Similarity Profiler

In [9]:
input = torch.randn(32, 3, 32, 32)

Conv2dNN_9_all_similarity.to('cpu')

with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True) as prof1:
    with record_function("model_inference"):
        Conv2dNN_9_all_similarity(input)

print(prof1.key_averages().table(sort_by="cpu_time_total"))

-------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      model_inference        -0.21%    -149.000us       100.00%      70.859ms      70.859ms      44.38 Mb     -35.81 Mb             1  
                         aten::conv1d         0.02%      13.000us        54.23%      38.430ms      19.215ms       6.00 Mb           0 b             2  
                    aten::convolution         0.12%      84.000us        54.22%      38.417ms      19.209ms       6.00 Mb           0 b             2  
                   aten::_convolution         0.63%     447.000us        54.10%      38.

STAGE:2025-02-09 13:43:42 4485:152408 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-09 13:43:42 4485:152408 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-09 13:43:42 4485:152408 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
